In [ ]:
import os
import torch as tf
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TORCHINDUCTOR_FX_GRAPH_CACHE"] = "1"
os.environ["TORCHINDUCTOR_AUTOGRAD_CACHE"] = "1"

In [ ]:
from llama_experiments import HORSE_DISTRIBUTION
from llama_models import tokenizer
from compute_batch_llama_epsilon_level_sets import learn_embeddings, BatchArgs
import scipy.stats 
import numpy as np

all_args = {}
results = {}

args = BatchArgs(max_iters = 500,
        learning_rate = 1e-3,
        epsilon = 1e-6,
        input_embeds = randomly_sampled_input_embeds,
        target_probabilities=np.tile(HORSE_DISTRIBUTION, (16,1))
)

result = learn_embeddings(args)

In [ ]:
all_norms = []
for key in results:
    learned_embeddings = results[key].inputs_embeds
    initial_embeddings = all_args[key].inputs_embeds
    
    
    norms = np.linalg.norm(learned_embeddings - initial_embeddings, ord = 2, axis = -1)
    norms = norms.flatten()
    norms = norms[norms > 0]
    all_norms.extend(norms.tolist())

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
sns.kdeplot(all_norms)
plt.title("Distribution of Δθ = ‖θ_final − θ_init‖")
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from llama_models import np_vocab_embedding_no_special_tokens


print(type(np_vocab_embedding_no_special_tokens), np_vocab_embedding_no_special_tokens.shape)
idxs_1 = np.random.choice(list(range(len(np_vocab_embedding_no_special_tokens))), 1000)
idxs_2 = np.random.choice(list(range(len(np_vocab_embedding_no_special_tokens))), 1000)
random_vocab_pair_norms = np.linalg.norm(np_vocab_embedding_no_special_tokens[idxs_1] - np_vocab_embedding_no_special_tokens[idxs_2], ord = 2, axis = -1)

sns.kdeplot(random_vocab_pair_norms)
plt.title("Δθ = ‖θⱼ − θᵢ‖₂")
plt.show()


In [ ]:
import seaborn as sns

diff = (learned_embeddings - initial_embeddings)
diff = diff.reshape(-1,diff.shape[-1])
norms = np.linalg.norm(learned_embeddings - initial_embeddings, ord = 2, axis = -1)
norms = norms.flatten()
diff = diff[norms > 0]
plt.title("Per-Dimension Displacement")

sns.kdeplot(diff.flatten())
plt.show()
